# Lab 9

In this lab, you will explore spatial data analysis using Python and DuckDB. You'll work with real-world datasets, ranging from global country statistics to specific building datasets. This will give you a practical understanding of handling, analyzing, and visualizing spatial data.

**Submission requirements**

1. **HTML Version:** Submit an HTML version of your notebook. Ensure all code outputs are visible. (Export via VS Code: Notebook > Export > HTML).
2. **Colab Link:** Provide a link to your notebook hosted on Google Colab for interactive review.

## Setup

Ensure you have DuckDB and Leafmap installed. Run the following command if needed:

In [2]:
# %pip install duckdb leafmap

In [3]:
import duckdb
import leafmap

## Question 1

Connect to a duckdb database and install the `httpfs` and `spatial` extensions

In [4]:
con= duckdb.connect() # defaults is memory 
con.install_extension('httpfs')
con.load_extension('httpfs')

In [5]:
con.install_extension('spatial')
con.load_extension('spatial')

## Question 2

Download the [Admin 0 – Countries](https://www.naturalearthdata.com/downloads/10m-cultural-vectors/) vector dataset from Natural Earth using the `leafmap.download_file()` function.

In [6]:
url= 'https://www.naturalearthdata.com/http//www.naturalearthdata.com/download/10m/cultural/ne_10m_admin_0_countries.zip'
#leafmap.download_file(url)

## Question 3

Create a new table in your database called `countries` and load the data from the downloaded country shapefile into it.

In [7]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS countries AS 
        SELECT * FROM ST_Read('ne_10m_admin_0_countries.shp')
        """
)

In [8]:
con.table('countries')

┌─────────────────┬───────────┬───────────┬───┬──────────────┬────────────────────┬──────────────────────┐
│   featurecla    │ scalerank │ LABELRANK │ … │  FCLASS_BD   │     FCLASS_UA      │         geom         │
│     varchar     │   int32   │   int32   │   │   varchar    │      varchar       │       geometry       │
├─────────────────┼───────────┼───────────┼───┼──────────────┼────────────────────┼──────────────────────┤
│ Admin-0 country │         0 │         2 │ … │ NULL         │ NULL               │ MULTIPOLYGON (((11…  │
│ Admin-0 country │         0 │         3 │ … │ NULL         │ NULL               │ MULTIPOLYGON (((11…  │
│ Admin-0 country │         0 │         2 │ … │ NULL         │ NULL               │ MULTIPOLYGON (((-6…  │
│ Admin-0 country │         0 │         3 │ … │ NULL         │ NULL               │ POLYGON ((-69.5100…  │
│ Admin-0 country │         0 │         2 │ … │ NULL         │ NULL               │ MULTIPOLYGON (((-6…  │
│ Admin-0 country │         0 │      

Calculate the total population of all countries in the database using the `POP_EST` column.

In [9]:
con.sql('SELECT SUM(POP_EST) AS total_pop FROM countries;')

┌──────────────┐
│  total_pop   │
│    double    │
├──────────────┤
│ 7677059722.3 │
└──────────────┘

Show the top 10 countries with the largest population.

In [10]:
con.sql(
    """
        SELECT SOVEREIGNT, POP_EST FROM countries
        ORDER BY POP_EST DESC
        LIMIT 10;
        """
)

┌──────────────────────────┬──────────────┐
│        SOVEREIGNT        │   POP_EST    │
│         varchar          │    double    │
├──────────────────────────┼──────────────┤
│ China                    │ 1397715000.0 │
│ India                    │ 1366417754.0 │
│ United States of America │  328239523.0 │
│ Indonesia                │  270625568.0 │
│ Pakistan                 │  216565318.0 │
│ Brazil                   │  211049527.0 │
│ Nigeria                  │  200963599.0 │
│ Bangladesh               │  163046161.0 │
│ Russia                   │  144373535.0 │
│ Mexico                   │  127575529.0 │
├──────────────────────────┴──────────────┤
│ 10 rows                       2 columns │
└─────────────────────────────────────────┘

Select countries in Europe with a population greater than 10 million and order them by population in descending order.

In [11]:
con.sql(
    """
        SELECT * FROM countries
        WHERE POP_EST > 10000000 AND REGION_UN = 'Europe'
        ORDER BY POP_EST DESC;
        """
)

┌─────────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────────────────┐
│   featurecla    │ scalerank │ LABELRANK │ … │ FCLASS_SE │ FCLASS_BD │ FCLASS_UA │         geom         │
│     varchar     │   int32   │   int32   │   │  varchar  │  varchar  │  varchar  │       geometry       │
├─────────────────┼───────────┼───────────┼───┼───────────┼───────────┼───────────┼──────────────────────┤
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((87…  │
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((13…  │
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((-5…  │
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((-7…  │
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((7.…  │
│ Admin-0 country │         0 │      

Save the results of the previous query as a new table called `europe`.

In [12]:
con.sql(
    """
        CREATE TABLE IF NOT EXISTS europe AS 
        SELECT * FROM countries
        WHERE POP_EST > 10000000 AND REGION_UN = 'Europe'
        ORDER BY POP_EST DESC;
        """
)

In [13]:
con.table('europe')

┌─────────────────┬───────────┬───────────┬───┬───────────┬───────────┬───────────┬──────────────────────┐
│   featurecla    │ scalerank │ LABELRANK │ … │ FCLASS_SE │ FCLASS_BD │ FCLASS_UA │         geom         │
│     varchar     │   int32   │   int32   │   │  varchar  │  varchar  │  varchar  │       geometry       │
├─────────────────┼───────────┼───────────┼───┼───────────┼───────────┼───────────┼──────────────────────┤
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((87…  │
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((13…  │
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((-5…  │
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((-7…  │
│ Admin-0 country │         0 │         2 │ … │ NULL      │ NULL      │ NULL      │ MULTIPOLYGON (((7.…  │
│ Admin-0 country │         0 │      

Export the `europe` table as a GeoJSON file.

In [14]:
# Add your code here

## Question 4

Create a table called `text_zones` and load the data from the [taxi_zones.parquet](https://beta.source.coop/cholmes/nyc-taxi-zones/taxi_zones.parquet) into it.

In [16]:
url= 'https://data.source.coop/cholmes/nyc-taxi-zones/taxi_zones.parquet'


In [18]:
con.sql(f"FROM '{url}'")

┌──────────┬────────────────┬────────────────┬───┬────────────┬───────────────┬──────────────────────┐
│ OBJECTID │   Shape_Leng   │   Shape_Area   │ … │ LocationID │    borough    │       geometry       │
│  int32   │ decimal(19,11) │ decimal(19,11) │   │   int32    │    varchar    │         blob         │
├──────────┼────────────────┼────────────────┼───┼────────────┼───────────────┼──────────────────────┤
│        1 │  0.11635745319 │  0.00078230679 │ … │          1 │ EWR           │ \x01\x03\x00\x00\x…  │
│        2 │  0.43346966679 │  0.00486634038 │ … │          2 │ Queens        │ \x01\x06\x00\x00\x…  │
│        3 │  0.08434110590 │  0.00031441416 │ … │          3 │ Bronx         │ \x01\x03\x00\x00\x…  │
│        4 │  0.04356652709 │  0.00011187195 │ … │          4 │ Manhattan     │ \x01\x03\x00\x00\x…  │
│        5 │  0.09214648986 │  0.00049795749 │ … │          5 │ Staten Island │ \x01\x03\x00\x00\x…  │
│        6 │  0.15049054252 │  0.00060646098 │ … │          6 │ Staten Is

In [21]:
con.sql(
    f"""
CREATE or REPLACE TABLE text_zones AS
SELECT * EXCLUDE geometry, ST_GeomFromWKB(geometry) AS geometry FROM
        '{url}'
"""
)

In [22]:
con.table('text_zones')

┌──────────┬────────────────┬────────────────┬───┬────────────┬───────────────┬──────────────────────┐
│ OBJECTID │   Shape_Leng   │   Shape_Area   │ … │ LocationID │    borough    │       geometry       │
│  int32   │ decimal(19,11) │ decimal(19,11) │   │   int32    │    varchar    │       geometry       │
├──────────┼────────────────┼────────────────┼───┼────────────┼───────────────┼──────────────────────┤
│        1 │  0.11635745319 │  0.00078230679 │ … │          1 │ EWR           │ POLYGON ((933100.9…  │
│        2 │  0.43346966679 │  0.00486634038 │ … │          2 │ Queens        │ MULTIPOLYGON (((10…  │
│        3 │  0.08434110590 │  0.00031441416 │ … │          3 │ Bronx         │ POLYGON ((1026308.…  │
│        4 │  0.04356652709 │  0.00011187195 │ … │          4 │ Manhattan     │ POLYGON ((992073.4…  │
│        5 │  0.09214648986 │  0.00049795749 │ … │          5 │ Staten Island │ POLYGON ((935843.3…  │
│        6 │  0.15049054252 │  0.00060646098 │ … │          6 │ Staten Is

Find out the unique values in the `borough` column and order them alphabetically.

In [ ]:
# Add your code here

Export the `text_zones` table as a parquet file.

In [ ]:
# Add your code here

## Question 5

Explore the [Google Open Buildings](https://beta.source.coop/cholmes/google-open-buildings/v2/geoparquet-admin1/) and select a country of your choice with relatively small number of buildings (i.e., small file size). Get the three character country code and replace `[COUNTRY_NAME]` in the following path with the country code. Use it to load all the parquet files for the selected country into a new table called `buildings`.

`s3://us-west-2.opendata.source.coop/google-research-open-buildings/v2/geoparquet-admin1/country=[COUNTRY_NAME]/*.parquet`

In [ ]:
# Add your code here

Find out the number of buildings in the selected country.

In [ ]:
# Add your code here

Find out the total area of all buildings in the selected country.

In [ ]:
# Add your code here

Export the `buildings` table as a GeoPackage file.

In [ ]:
# Add your code here